In [1]:
import os
import random
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
import warnings

import wandb
from otc.metrics.metrics import effective_spread
from scipy.stats import wilcoxon
from tqdm.auto import tqdm


In [2]:
# set here globally
EXCHANGE = "ise"
MODELS = ["gbm", "classical"] 
SUBSET = "test"  # "all"
STRATEGY = "supervised" # "transfer" 


In [3]:
KEY = f"{EXCHANGE}_{STRATEGY}_{SUBSET}"
DATASET = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(DATASET)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in MODELS:
    results = f"fbv/thesis/{EXCHANGE}_{model}_{STRATEGY}_{SUBSET}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact ise_supervised_raw:latest, 2589.45MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact ise_gbm_supervised_test:latest, 141.58MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact ise_classical_supervised_test:latest, 499.12MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [4]:
# p. 35-38
COLUMNS = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if SUBSET == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=COLUMNS
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=COLUMNS
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif SUBSET == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
    )


results = []
for i, model in tqdm(enumerate(MODELS)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=MODELS)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


0it [00:00, ?it/s]

In [5]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        ("gbm", "gbm(classical)"),
        # ("gbm", "gbm(classical-retraining)"),
        ("gbm", "gbm(classical-size)"),
        # ("gbm", "gbm(classical-size-retraining)"),
        ("gbm", "gbm(ml)"),
        # ("gbm", "gbm(ml-retraining)"),
        # ("gbm", "gbm(semi-classical)"),
        # ("gbm",'gbm(semi-classical-retraining)'),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        ("classical", "quote(best)->quote(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)",
        ),
    ]
]


In [6]:
results_data


gbm                             classical            \
         gbm(classical) gbm(classical-size) gbm(ml)  tick(ex) quote(ex)   
index                                                                     
39342191             -1                  -1      -1         1        -1   
39342190             -1                  -1      -1        -1        -1   
39342189              1                   1       1         1         1   
39342188             -1                  -1      -1        -1        -1   
39342187             -1                  -1      -1         1        -1   
...                 ...                 ...     ...       ...       ...   
49203742             -1                   1      -1        -1         1   
49203743              1                   1       1        -1         1   
49203744              1                   1       1        -1        -1   
49203745              1                   1       1        -1         1   
49203746              1                   1       1         1         1   

                                                         \
         lr(ex) emo(ex) clnv(ex) quote(best)->quote(ex)   
index                                                     
39342191     -1      -1       -1                     -1   
39342190     -1      -1       -1                     -1   
39342189      1       1        1                      1   
39342188     -1      -1       -1                     -1   
39342187     -1      -1       -1                     -1   
...         ...     ...      ...                    ...   
49203742      1       1        1                      1   
49203743      1      -1        1                      1   
49203744     -1      -1       -1                     -1   
49203745      1       1        1                      1   
49203746      1       1        1                      1   

                                                                                        
         trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)  
index                                                                                   
39342191                                                 -1                             
39342190                                                 -1                             
39342189                                                  1                             
39342188                                                 -1                             
39342187                                                 -1                             
...                                                     ...                             
49203742                                                  1                             
49203743                                                  1                             
49203744                                                 -1                             
49203745                                                  1                             
49203746                                                  1                             

[9861576 rows x 10 columns]

In [7]:
LUT = {
    "Trade_Size(ex)->Quote(Best)->Depth(Best)->Quote(Ex)->Depth(Ex)->Rev_Tick(All)": "\gls{GBM}",
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Location to Quote",
    "all": "All trades",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [8]:
def set_tex_style(styler, caption, label, bold_axis=1):
    res = styler.set_caption(caption)

    res = (
        res.apply(highlight_max, props="font-weight:bold;", axis=bold_axis)
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [9]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


## Unclassified by method

In [10]:
results_data.head()


gbm                             classical            \
         gbm(classical) gbm(classical-size) gbm(ml)  tick(ex) quote(ex)   
index                                                                     
39342191             -1                  -1      -1         1        -1   
39342190             -1                  -1      -1        -1        -1   
39342189              1                   1       1         1         1   
39342188             -1                  -1      -1        -1        -1   
39342187             -1                  -1      -1         1        -1   

                                                         \
         lr(ex) emo(ex) clnv(ex) quote(best)->quote(ex)   
index                                                     
39342191     -1      -1       -1                     -1   
39342190     -1      -1       -1                     -1   
39342189      1       1        1                      1   
39342188     -1      -1       -1                     -1   
39342187     -1      -1       -1                     -1   

                                                                                        
         trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)  
index                                                                                   
39342191                                                 -1                             
39342190                                                 -1                             
39342189                                                  1                             
39342188                                                 -1                             
39342187                                                 -1

In [11]:
unclassified = (
    (results_data[results_data == 0.0].count(axis=0) / len(results_data.index))
    .sort_values(ascending=False)
    .to_frame(name="unclassified")
)


In [12]:
unclassified.style.pipe(
    set_tex_style,
    caption=(f"{KEY}-unclassified-long", "{key}-unclassified-short"),
    label=f"{KEY.lower()}-unclassfied",
    bold_axis=0,
)
unclassified


unclassified
classical tick(ex)                                                0.124762
          quote(ex)                                               0.060195
          quote(best)->quote(ex)                                  0.037738
          emo(ex)                                                 0.023059
          clnv(ex)                                                0.019444
          lr(ex)                                                  0.001618
          trade_size(ex)->quote(best)->quote(ex)->depth(b...      0.000043
gbm       gbm(classical)                                          0.000000
          gbm(classical-size)                                     0.000000
          gbm(ml)                                                 0.000000

## Fill in unclassified

Unclassified are `0`.

In [13]:
rng = np.random.default_rng(42)

# replace 0 -> nan -> [-1,1]
results_data.replace(0, np.nan, inplace=True)
filler = pd.DataFrame(
    rng.choice(a=[-1, 1], size=results_data.shape),
    index=results_data.index,
    columns=results_data.columns,
)
results_data.fillna(filler, inplace=True)


### Robustness Checks

In [14]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "<= 1",
    "(1-2]",
    "(2-3]",
    "(3-6]",
    "(6-12]",
    "> 12",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock option",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "<= 0.7",
    "(0.7-0.9]",
    "(0.9-1.1]",
    "(1.1-1.3]",
    "> 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

# require ask >= bid
mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        (bid < trade_price) & (trade_price < ask),
        1,  # inside
        np.where(
            (trade_price == bid) | (ask == trade_price),
            2,  # at quotes
            np.where((trade_price < bid) | (ask < trade_price), 3, 4),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print["all"] = "all"

X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "year",
    ],
    inplace=True,
)


In [15]:
X_print = pd.concat([X_print, results_data], axis=1)


In [16]:
X_print.head().T


index,39342191,39342190,39342189,39342188,39342187
buy_sell,-1,-1,1,-1,-1
TRADE_PRICE,3.5,6.38,0.13,0.04,0.4
ask_ex,3.85,6.8,0.13,0.1,0.55
bid_ex,3.5,6.35,0.07,0.04,0.31
OPTION_TYPE,P,C,P,C,C
issue_type,Stock option,Stock option,Others,Others,Others
TRADE_SIZE_binned,"(3,5]","(0,1]","(0,1]","(1,3]","(3,5]"
year_binned,2015,2015,2015,2015,2015
ttm_binned,<= 1,<= 1,<= 1,<= 1,<= 1
myn_binned,(0.9-1.1],(0.9-1.1],(0.7-0.9],(0.7-0.9],(0.9-1.1]


## Accuracy Calculation

In [17]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
        #         acc_tot = accuracy_score(
        #             X_print["buy_sell"].astype("int8"), X_print[classifier]
        #         )

        #         res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{KEY.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [19]:
master


gbm                                 \
                               gbm(classical) gbm(classical-size)    gbm(ml)   
Option Type       C                 62.889592           71.895259  73.624730   
                  P                 64.544991           72.892594  74.593715   
Security Type     Index option      56.308184           57.640799  58.100126   
                  Others            68.405530           76.385942  77.551217   
                  Stock option      61.866828           70.962294  72.919301   
Trade Size        (0,1]             61.886324           72.589837  74.242395   
                  (1,3]             62.455088           73.006333  74.813673   
                  (3,5]             62.683294           72.766890  74.557255   
                  (5,11]            65.941513           71.771961  73.383853   
                  >11               67.305363           71.307368  73.084679   
Year              2015              60.396806           69.028053  71.423694   
                  2016              63.737864           72.560975  74.235176   
                  2017              64.587031           73.018742  74.603733   
Time to Maturity  <= 1              64.456091           72.770627  74.560803   
                  (1-2]             64.596545           72.885430  73.865438   
                  (2-3]             62.979868           71.797757  72.946987   
                  (3-6]             61.803170           71.302394  72.719581   
                  (6-12]            61.849926           71.222635  73.093080   
                  > 12              55.148148           68.602077  72.039005   
Moneyness         <= 0.7            65.551797           71.843131  72.438812   
                  (0.7-0.9]         67.636359           74.290189  75.357081   
                  (0.9-1.1]         64.063302           72.990099  74.414466   
                  (1.1-1.3]         54.203956           66.292506  70.268926   
                  > 1.3             52.572955           63.007743  70.550826   
Location to Quote at mid            62.620494           72.155374  74.659713   
                  inside            62.608299           68.336275  70.249568   
                  at quotes         67.805369           86.648398  87.273077   
                  outside           60.497886           60.377105  63.067839   
                  unknown           78.583647           78.371536  79.131030   
All trades        all               63.662370           72.360838  74.077074   

                                classical                                   \
                                 tick(ex)  quote(ex)     lr(ex)    emo(ex)   
Option Type       C             50.348015  56.338657  56.416520  53.500982   
                  P             50.101214  57.748410  57.797980  54.118193   
Security Type     Index option  50.893608  53.728014  53.788502  51.280161   
                  Others        49.868438  62.402034  62.494080  57.510196   
                  Stock option  50.369423  54.870101  54.923804  52.328385   
Trade Size        (0,1]         50.291338  55.288035  55.418330  51.930230   
                  (1,3]         50.207361  55.312322  55.398067  51.948962   
                  (3,5]         49.630289  55.723949  55.768923  52.821732   
                  (5,11]        50.145873  59.893257  59.899878  57.168629   
                  >11           50.757787  60.714690  60.695406  57.231189   
Year              2015          50.344807  54.902913  54.892890  52.851584   
                  2016          50.172148  57.571543  57.635410  54.270965   
                  2017          50.335905  56.367496  56.459133  52.986116   
Time to Maturity  <= 1          50.118387  57.301220  57.362873  54.417911   
                  (1-2]         50.223068  57.744957  57.762882  53.547187   
                  (2-3]         50.502421  57.086186  57.179021  52.890993   
                  (3-6]         50.762062  56.015521  56.110247  52.001021   

In [20]:
master.style.pipe(
    set_tex_style,
    caption=("master-long", "master-short"),
    label=f"{KEY}-master",
    bold_axis=0,
)


## Effective Spread 💴

In [21]:
X_print.head()

,buy_sell,TRADE_PRICE,ask_ex,bid_ex,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,...,"(gbm, gbm(classical))","(gbm, gbm(classical-size))","(gbm, gbm(ml))","(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, quote(best)->quote(ex))","(classical, trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all))"
index,,,,,,,,,,,,,,,,,,,,,
39342191,-1,3.50,3.85,3.50,P,Stock option,"(3,5]",2015,<= 1,(0.9-1.1],...,-1,-1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342190,-1,6.38,6.80,6.35,C,Stock option,"(0,1]",2015,<= 1,(0.9-1.1],...,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342189,1,0.13,0.13,0.07,P,Others,"(0,1]",2015,<= 1,(0.7-0.9],...,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0
39342188,-1,0.04,0.10,0.04,C,Others,"(1,3]",2015,<= 1,(0.7-0.9],...,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342187,-1,0.40,0.55,0.31,C,Others,"(3,5]",2015,<= 1,(0.9-1.1],...,-1,-1,-1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [22]:
results = []

ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
mid = X_print["mid"]

# calculate true rel effective spread but not aggregated, convert to %
es_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], mid, mode="none")
eps_true = np.empty(es_true.shape)
np.divide(es_true, mid, out=eps_true, where=mid != 0)

nom_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], mid, mode="nominal")
rel_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], mid, mode="relative")

# require ask > bid
rel_quoted = np.nanmean((ask - bid) / mid)
nom_quoted = np.nanmean(np.where(ask >= bid, (ask - bid), np.nan))

for classifier in tqdm(classifiers):
    nom_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], mid, mode="nominal")
    rel_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], mid, mode="relative")

    # calculate pred rel effective spread but not aggregated convert to %
    es_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], mid, mode="none")
    eps_pred = np.empty(es_pred.shape)
    np.divide(es_pred, mid, out=eps_pred, where=mid != 0)

    wilcoxon_res  = wilcoxon(eps_pred, eps_true, nan_policy="omit", zero_method="zsplit")

    res = pd.Series(
            {
                "nom_pred": nom_pred * 100,
                "rel_pred": rel_pred * 100,
                "statistic":wilcoxon_res.statistic,
                "pvalue":wilcoxon_res.pvalue,
            }, name=classifier
        )
    results.append(res)

true_eff = pd.Series({"nom_pred":nom_true * 100, "rel_pred": rel_true * 100, "statistic":np.NaN, "pvalue":np.NaN}, name="true_eff")
true_quoted = pd.Series({"nom_pred":nom_quoted * 100, "rel_pred": rel_quoted * 100, "statistic":np.NaN, "pvalue":np.NaN}, name="true_quoted")

results.append(true_eff)
results.append(true_quoted)

results = pd.concat(results, axis=1)

  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
results.T.style.format("{:.3f}")


,nom_pred,rel_pred,statistic,pvalue
"('gbm', 'gbm(classical)')",7.414,9.186,15449198918565.000,0.000
"('gbm', 'gbm(classical-size)')",4.247,6.983,18996716578607.500,0.000
"('gbm', 'gbm(ml)')",3.741,6.513,19613712891116.500,0.000
"('classical', 'tick(ex)')",1.557,1.073,22593375152463.500,0.000
"('classical', 'quote(ex)')",16.333,16.207,9330692653233.000,0.000
"('classical', 'lr(ex)')",16.333,16.207,9330692653233.000,0.000
"('classical', 'emo(ex)')",4.647,8.443,21566549392574.500,0.000
"('classical', 'clnv(ex)')",11.626,13.284,18544330865464.000,0.000
"('classical', 'quote(best)->quote(ex)')",6.271,9.419,15830405879950.500,0.000
"('classical', 'trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)')",1.415,4.293,20692020255975.000,0.000


In [24]:
results.style.to_latex(
    f"../reports/Content/{KEY}-eff-spread.tex",
    siunitx=True,
    position_float="centering",
    hrules=True,
    clines="skip-last;data",
    label=f"tab:eff-{KEY}",
    caption=(f"long-eff-{KEY}", f"short-eff-{KEY}"),
    convert_css=True,
)


## Diffs 🔄️

In [25]:
# classical baselines

base = master[
    [
        ("classical", "quote(best)->quote(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)",
        ),
        (
            "classical",
            "trade_size(ex)->quote(best)->quote(ex)->depth(best)->depth(ex)->rev_tick(all)",
        ),
    ]
]

# my ml models
revised = master[
    [("gbm", "gbm(classical)"), ("gbm", "gbm(classical-size)"), ("gbm", "gbm(ml)")]
]


In [26]:
def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns.get_level_values(1)
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    for i, mul_col in enumerate(combo.columns):

        combo[(mul_col[0], "pm")] = (combo[mul_col] - base.iloc[:, i]).round(2)
        combo.sort_index(axis=1, inplace=True)

    return combo


In [27]:
diff = combine_results(revised, base)

diff.style.to_latex(
    f"../reports/Content/diff-{KEY}.tex",
    siunitx=True,
    position_float="centering",
    hrules=True,
    clines="skip-last;data",
    label=f"tab:diff-{KEY}",
    caption=(f"long-diff-{KEY}", f"short-diff-{KEY}"),
    convert_css=True,
)


In [28]:
diff


gbm(classical)       gbm(classical-size)        \
                                          nom    pm                 nom    pm   
Option Type       C                 62.889592  4.01           71.895259  4.90   
                  P                 64.544991  3.77           72.892594  5.45   
Security Type     Index option      56.308184 -1.49           57.640799 -0.88   
                  Others            68.405530  3.16           76.385942  6.25   
                  Stock option      61.866828  4.28           70.962294  4.81   
Trade Size        (0,1]             61.886324  3.67           72.589837  3.81   
                  (1,3]             62.455088  4.28           73.006333  4.16   
                  (3,5]             62.683294  4.20           72.766890  3.81   
                  (5,11]            65.941513  3.65           71.771961  8.48   
                  >11               67.305363  3.88           71.307368  6.76   
Year              2015              60.396806  4.25           69.028053  5.68   
                  2016              63.737864  3.84           72.560975  5.44   
                  2017              64.587031  3.91           73.018742  4.32   
Time to Maturity  <= 1              64.456091  3.88           72.770627  5.66   
                  (1-2]             64.596545  4.05           72.885430  4.77   
                  (2-3]             62.979868  3.26           71.797757  3.91   
                  (3-6]             61.803170  3.93           71.302394  3.85   
                  (6-12]            61.849926  4.17           71.222635  3.75   
                  > 12              55.148148  4.21           68.602077  3.90   
Moneyness         <= 0.7            65.551797  4.10           71.843131  7.73   
                  (0.7-0.9]         67.636359  4.00           74.290189  6.42   
                  (0.9-1.1]         64.063302  3.83           72.990099  4.84   
                  (1.1-1.3]         54.203956  4.18           66.292506  4.63   
                  > 1.3             52.572955  3.85           63.007743  3.03   
Location to Quote at mid            62.620494  6.67           72.155374  6.25   
                  inside            62.608299  2.40           68.336275  4.09   
                  at quotes         67.805369  7.84           86.648398  8.44   
                  outside           60.497886 -6.11           60.377105 -4.52   
                  unknown           78.583647  2.24           78.371536  1.93   
All trades        all               63.662370  3.90           72.360838  5.16   

                                  gbm(ml)         
                                      nom     pm  
Option Type       C             73.624730   6.63  
                  P             74.593715   7.15  
Security Type     Index option  58.100126  -0.42  
                  Others        77.551217   7.41  
                  Stock option  72.919301   6.77  
Trade Size        (0,1]         74.242395   5.46  
                  (1,3]         74.813673   5.97  
                  (3,5]         74.557255   5.60  
                  (5,11]        73.383853  10.09  
                  >11           73.084679   8.54  
Year              2015          71.423694   8.07  
                  2016          74.235176   7.12  
                  2017          74.603733   5.90  
Time to Maturity  <= 1          74.560803   7.45  
                  (1-2]         73.865438   5.75  
                  (2-3]         72.946987   5.06  
                  (3-6]         72.719581   5.27  
                  (6-12]        73.093080   5.62  
                  > 12          72.039005   7.33  
Moneyness         <= 0.7        72.438812   8.33  
                  (0.7-0.9]     75.357081   7.49  
                  (0.9-1.1]     74.414466   6.26  
                  (1.1-1.3]     70.268926   8.61  
                  > 1.3         70.550826  10.58  
Location to Quote at mid        74.659713   8.76  
                  inside        70.249568   6.01  
          